# Explore Wavelet Transform with pywt

In order to acquire a good theoretical foundation on the topic of wavelet transform, I found useful information here: 
    
    https://www.youtube.com/watch?v=i0rPaAXjJoI

    Rubin H. Landau et al., 2015, Computational Physics (third edition), Wiley-VCH, pp. 307-319.

I found it helpful to familiarize myself with the Concept of Fourier Transform first. I found useful information here: 
    
    https://www.youtube.com/watch?v=spUNpyF58BY
        
    James H. McClellan et al., 2003, Signal Processing First (International Edition), Pearson Education International, pp. 1-50.
    

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pylab
import pywt
#from intra_event_processing.data_processing.autocorrelation import autocorr_non_norm, autocorr_norm
#from intra_event_processing.data_visualization.show_dic_structure import show_structure

## Continuous wavelet transform: 

### Create some test data:

#### Two sine wave signals, the second with a frequency four times as high as the first one:

In [ ]:
f_1 = 4
w = 2. * np.pi * f_1
time_interval = 1
fig = pylab.figure()
signals_lowfreq = {}
for i, samples in enumerate((5, 14, 200, 5000)):
    pylab.subplot(2, 2, i+1)
    pylab.title('%i samples'%samples)
    t = np.linspace(0, time_interval, samples)
    y = np.sin(w * t)
    data_points = str(samples) + '_points'
    signals_lowfreq[data_points] = y
    pylab.plot(t, y, '.-')
    plt.subplots_adjust(bottom=0.4, right=2, top=2)
fig.show()

In [ ]:
f_2 = f_1*4
w = 2. * np.pi * f_2
time_interval = 1
fig = pylab.figure()
signals_highfreq = {}
for i, samples in enumerate((5, 14, 200, 5000)):
    pylab.subplot(2, 2, i+1)
    pylab.title('%i samples'%samples)
    t = np.linspace(0, time_interval, samples)
    y = np.sin(w * t)
    data_points = str(samples) + '_points'
    signals_highfreq[data_points] = y
    pylab.plot(t, y, '.-')
    plt.subplots_adjust(bottom=0.4, right=2, top=2)
fig.show()

In [ ]:
sine_wave_low = signals_lowfreq['200_points']
sine_wave_high = signals_highfreq['200_points']

In [ ]:
pylab.subplot(2,2,1)
pylab.plot(sine_wave_low)
pylab.subplot(2,2,2)
pylab.plot(sine_wave_high)
fig.show()

#### A mixed sine wave signal: sum up two frequencies

In [ ]:
signal_mixed_total = signals_lowfreq['200_points'] + signals_highfreq['200_points']

In [ ]:
plt.plot(signal_mixed_total)

#### Constant zero signal and a concatenated signal: First part constant zero, second part high frequency sine wave:

In [ ]:
zero_streak = np.zeros(100)
zero_high = np.concatenate((zero_streak,signals_highfreq['200_points'][100:]))
len(zero_high)

In [ ]:
pylab.subplot(2,2,1)
pylab.plot(zero_streak)
pylab.subplot(2,2,2)
pylab.plot(zero_high)
fig.show()

#### Concatenated two parts signal: Low frequency and mixed frequency (low and high) sine wave

In [ ]:
signal_mixed_half = signals_lowfreq['200_points'] + zero_high

In [ ]:
plt.plot(signal_mixed_half)

#### Visualize gaussian wavelet: 

In [ ]:
w = 'gaus1'
w = pywt.ContinuousWavelet(w)
dir(w)
for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

### Applying continuous wavelet transform to the slow sine wave signal:

#### Specify the imput data, the scales and the wavelet type:

Scales define how much the wavelet will be streched or squished: It affects frequency in the following way: freq = 2*pi/s

In [ ]:
scales = np.arange(1, 11)
#scales = np.arange(1,3)

In [ ]:
scales

#### Execute wavelet transform: 

In [ ]:
coeffs_1, freqs_1 = pywt.cwt(sine_wave_low, scales, 'gaus1')

Two outputs are produced: An array of the different frequencies defined by the different scales (number of output frequencies equals number of input scales), and array of coefficient arrays. Each coefficient array corresponds one wavelet frequency and scale (number of arrays equal numbers of input scales). Each of these coefficient-arrays contains coefficients, the number of coefficients in each array equals the number of data points in the input data. Each coefficient corresponds to a tau value indicating the location of the wavelet and the coefficient value itself is a measure of the similarity of the signal and the wavelet at that specific location (obtained by convolution of that portion of the signal with the wavelet). 

In [ ]:
type(freqs_1)

In [ ]:
print(len(scales))
print(len(coeffs_1))
print(len(freqs_1))

In [ ]:
print(len(coeffs_1[0]))
print(len(sine_wave_low))

#### Visualize results: 

The graph below shows the frequency of the signal at different scales, i.e. the frequencies of the signal as revealed by convolution with the wavelet at different scales:

In [ ]:
plt.figure(1, figsize=(20, 10))
plt.subplot(121)
plt.imshow(coeffs_1, cmap='coolwarm', aspect='auto')
plt.show()

Plotting an coefficient array reveales a similar pattern as the input signal. This makes sense as the wavelet has a sine shape and the convolution with the signal yields the highest values as the wavelet is in phase with the signal: 

In [ ]:
plt.plot(coeffs_1[0])

Plotting the frequencies shows a decrease. This makes sense as the arrays of frequencies corresponds to an array of increasing scales.

In [ ]:
plt.plot(freqs_1)

### Applying continuous wavelet transform to the concatenated two-part-signal: low freq sine wave and mixed low-and-high-freq-sine-wave:

In [ ]:
coeffs_new_1, freqs_new_1 = pywt.cwt(signal_mixed_half, scales, 'gaus1')

#### Visualize results:

The plot below shows that the low frequency is present throughout the signal whereas the high frequency is only present in the second half. It also shows that the high frequency signal can only be revealed at lower scales (i.e. high frequency wavelets): 

In [ ]:
plt.figure(1, figsize=(20, 10))
plt.subplot(121)
plt.imshow(coeffs_new_1, cmap='coolwarm', aspect='auto')
plt.show()

The high scale coefficients capture the shape of the low freq sine wave (amplitude is higher because it is a convolution). The low frequency sine wave is present in the entire signal: 

In [ ]:
freqs_new_1[9]

In [ ]:
plt.plot(coeffs_new_1[9])

The low scale coefficients capture the shape of the high frequency sine wave. They are only present in the second half of the signal: 

In [ ]:
plt.plot(coeffs_new_1[0])

In [ ]:
freqs_new_1[0]

### Questions:

Why is the high frequency of the wavelet not at all visible in the first half of the coefficient trace? Convoluting a high frequency wave with a low frequency signal would not make the high frequency disappear completely, right? <br>
<br>
And why is the amplitude so much smaller than in the original signal?

### Possible answer to the above questions:
The integral of the wavelet is zero. Depending on the shape of the wavelet and the signal the convolution of the two can result to zero. Also, the convolution is normalized by dividing it by sqrt(s), s being the scale of the wavelet. Is this correct?

## Discrete wavelet transform: 

### Create different types of test data: 

In [ ]:
ecg = pywt.data.ecg()

data_constant = np.repeat(2, 1000)

data1 = np.arange(1, 1025)

data2 = np.concatenate((np.arange(1, 400),
                        np.arange(398, 600),
                        np.arange(601, 1024)))

data3 = np.concatenate((np.arange(1,300), np.arange(350,1000)))

x = np.linspace(0.082, 2.128, num=1024)[::-1]
data4 = np.sin(40 * np.log(x)) * np.sign((np.log(x)))



In [ ]:
#fig, axes = plt.subplots(nrows=3, ncols=2)
#fig.tight_layout()
fig = plt.figure(figsize=(10,10))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=1, hspace=0.5)
plt.subplot(3,2,1)
plt.title('Constant values')
plt.plot(data_constant)
plt.subplot(3,2,2)
plt.title('Constant slope')
plt.plot(data1)
plt.subplot(3,2,3)
plt.title('Constant slope with overlap at one point and a gap at a second point')
plt.plot(data2)
plt.subplot(3,2,4)
plt.title('Constant slope with sudden jump')
plt.plot(data3)
plt.subplot(3,2,5)
plt.title('Sine wave with increasing frequency and irregularity')
plt.plot(data4)
plt.subplot(3,2,6)
plt.title('ecg')
plt.plot(ecg)
fig.show()

In [ ]:
data2[599:603]

### Visualize the haar wavelet:

In [ ]:
w = 'haar'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()


### Apply wavelet transform:

In [ ]:
def dwt_steps(a, w, mode, num_steps):
    ca = []
    cd = []
    for i in range(num_steps + 1):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)
    return ca, cd

In [ ]:
"""Decompose and plot a signal S.
S = An + Dn + Dn-1 + ... + D1
"""

mode = pywt.Modes.smooth

ca_const, cd_const = dwt_steps(data_constant, w, mode, 4)
ca_1, cd_1 = dwt_steps(data1, w, mode, 4)
ca_2, cd_2 = dwt_steps(data2, w, mode, 4)
ca_3, cd_3 = dwt_steps(data3, w, mode, 4)
ca_4, cd_4 = dwt_steps(data4, w, mode, 4)

### Discrete convolution and discrete scaling steps:
Contrary to continuous wavelet transform, in discrete wavelet transform convolution of the wavelet with the signal is not performed by sliding the wavelet continuously along the signal. Instead, one can imagine wavelet copies being lined up along the signal one next to another. 
Scaling steps are discrete as well in that the length of the wavelet is doubled every round (at least in this concrete example). 

#### The Haar wavelet and constant values:
The haare wavelet detects sudden decreases (and will detect sudden increases as negative changes). 
Hence, convolution with a trace of constant values will yield a result of zero: 

In [ ]:
plt.plot(cd_const[0])

### Haar wavelet and constant slope:
Convolution with a constantly (linearly) increasing signal will yield a constant negative value.
Changes in the response are due to computer inacuracies and disappear when the numbers are rounded appropriately:

In [ ]:
plt.plot(cd_1[0])

In [ ]:
plt.plot(np.around(cd_1[0]))

### Haar wavelet and irregularities:
The Haar wavelet will detect sudden changes in an otherwise smooth linear signal such as repeating values, gaps or sudden jumps: 

In [ ]:
plt.plot(np.around(cd_2[0], 11))

In [ ]:
plt.plot(np.around(cd_3[0], 11))

In [ ]:
plt.plot(ca_3[0])

### Haar wavelet and sine wave:
The Haar wavelet will reveal changing slopes:

In [ ]:
plt.plot(cd_4[0])

### Details and approximation:
The cd object is a trace of all the convolution values. Since the wavelet was two data points long at its smalles scale, the first array of the cd object is half the length of the original signal:

In [ ]:
print(len(cd_1[0]))
print(len(data1))

At every iteration the scale of the wavelet is doubled and hence the length of the output is halfed:

In [ ]:
print(len(cd_1[1]))
print(len(cd_1[2]))
print(len(cd_1[3]))
print(len(cd_1[4]))

The ca object contains the convolution results of the low pass filter. The data trace is convoluted with the low-pass filter. The result is a down-sampled data trace with higher values. In other words, the signal is convoluted with a "step" wavelet (two consecutive values of 1), which means two consecutive values are picked up (multiplied by 1), then added together and divided by the square root of 2^j, where 2^j is the scale s that defines the frequency of the wavelet. Then the same procedure is repeated for the next two values and so on: 

In [ ]:
print(data1[0:6])
print(ca_1[0][0:3])
print(1/np.sqrt(2) + 2/np.sqrt(2))
print(3/np.sqrt(2) + 4/np.sqrt(2))
print(5/np.sqrt(2) + 6/np.sqrt(2))

The process explained above is repeated at every iteration: For scale s = 2^j, 2^j consecutive values in data1 are added up and then divided by sqrt(2^j). The result is then stored in ca_1[j-1].  So, for j=2, four consecutive values are added up and divided by 2:

In [ ]:
print(ca_1[0][0:4])
print(ca_1[1][0:4])
print(data1[0]/2 + data1[1]/2 + data1[2]/2 + data1[3]/2)
print(data1[4]/2 + data1[5]/2 + data1[6]/2 + data1[7]/2)
print(data1[8]/2 + data1[9]/2 + data1[10]/2 + data1[11]/2)

The same result is obtained by adding up 2^(j-1) values of ca_1[j-2] and dividing them by sqrt(2^(j-1): 

In [ ]:
print(ca_1[0][0]/np.sqrt(2) + ca_1[0][1]/np.sqrt(2))
print(ca_1[0][2]/np.sqrt(2) + ca_1[0][3]/np.sqrt(2))
print(ca_1[0][4]/np.sqrt(2) + ca_1[0][5]/np.sqrt(2))

In [ ]:
plt.plot(data1)

In [ ]:
plt.plot(ca_1[0])

The cd_1 object contains the results obtained by the convolution of the signal with the haar wavelet. The convolution is carried out - as was the convolution with the low-pass filter - first on the two consecutive values, then on the next two consecutive values, and so on: 

In [ ]:
print(cd_1[0][0:20])
print(data1[0]/np.sqrt(2) - data1[1]/np.sqrt(2))
print(data1[2]/np.sqrt(2) - data1[3]/np.sqrt(2))

Since the haar wavelet detects negative changes and the signal at hand shows positive change (constant positive slope i.e. a linearily increasing function), the convolution with the haar wavelet here yields a constant negative value: 

In [ ]:
plt.plot(np.around(cd_1[0][0:1000]))

The below code verifies the above explanation for ca_1[0] and cd_1[0]:

In [ ]:
index = 0
ca_values_check_1 = []
for iteration in range(1,(int(len(ca_1[0])/2))):
    convolution_value = ca_1[0][index]/np.sqrt(2) + ca_1[0][index + 1]/np.sqrt(2)
    ca_values_check_1.append(convolution_value)
    index = index + 2 

In [ ]:
index = 0
cd_values_check_1 = []
for iteration in range(1,(int(len(ca_1[0])/2))):
    convolution_value = ca_1[0][index]/np.sqrt(2) - ca_1[0][index + 1]/np.sqrt(2)
    cd_values_check_1.append(convolution_value)
    index = index + 2 

In [ ]:
np.array_equal(np.around(ca_values_check_1, 2)[0:len(ca_values_check_1)], np.around(ca_1[1], 2)[0:len(ca_values_check_1)])

In [ ]:
np.array_equal(np.around(cd_values_check_1, 2)[0:len(cd_values_check_1)], np.around(cd_1[1], 2)[0:len(cd_values_check_1)])

### Reconstruction step:

In [ ]:
rec_a_1 = []
rec_d_1 = []

for i, coeff in enumerate(ca_1):
    coeff_list = [coeff, None] + [None] * i
    rec_a_1.append(pywt.waverec(coeff_list, w))

for i, coeff in enumerate(cd_1):
    coeff_list = [None, coeff] + [None] * i
    rec_d_1.append(pywt.waverec(coeff_list, w))

In [ ]:
print(type(rec_a_1))
print(type(rec_d_1))
print(len(rec_a_1[4]))
print(len(rec_d_1[4]))
print(len(data1))

The pywt.dwt() function reconstructs a signal of the same length as the original signal from the ca object. The ca objects from early iterations (small scale wavelets) will yield a more complete reconstruction: 

In [ ]:
plt.plot(data1)

In [ ]:
plt.plot(rec_a_1[0])

In [ ]:
plt.plot(rec_a_1[4])

The pywt.dwt() function reconstructs the convolution of the original signal with wavelet. Plotting shows the convolution results represented as areas under the curve: 

In [ ]:
plt.plot(rec_d_1[4])

### The concepts explained above can be used to detect irregularities in different signals:

In [ ]:
fig = plt.figure(figsize=(20,10))
pylab.subplot(4,4,1)
pylab.plot(data_constant)
pylab.subplot(4,4,2)
pylab.plot(data1)
pylab.subplot(4,4,3)
pylab.plot(data2)
pylab.subplot(4,4,4)
pylab.plot(data3)
pylab.subplot(4,4,5)
pylab.plot(data4)
pylab.subplot(4,4,6)
pylab.plot(ecg)
fig.show()

In [ ]:


mode = pywt.Modes.smooth

def plot_signal_decomp(data, w, title):
    """Decompose and plot a signal S.
    S = An + Dn + Dn-1 + ... + D1
    """
    w = pywt.Wavelet(w)
    a = data
    ca = []
    cd = []
    for i in range(10):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)

    rec_a = []
    rec_d = []

    for i, coeff in enumerate(ca):
        coeff_list = [coeff, None] + [None] * i
        rec_a.append(pywt.waverec(coeff_list, w))

    for i, coeff in enumerate(cd):
        coeff_list = [None, coeff] + [None] * i
        rec_d.append(pywt.waverec(coeff_list, w))

    fig = plt.figure(figsize=(20,10))
    ax_main = fig.add_subplot(len(rec_a) + 1, 1, 1)
    ax_main.set_title(title)
    ax_main.plot(data)
    ax_main.set_xlim(0, len(data) - 1)

    for i, y in enumerate(rec_a):
        ax = fig.add_subplot(len(rec_a) + 1, 2, 3 + i * 2)
        ax.plot(y, 'r')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("A%d" % (i + 1))

    for i, y in enumerate(rec_d):
        ax = fig.add_subplot(len(rec_d) + 1, 2, 4 + i * 2)
        ax.plot(y, 'g')
        ax.set_xlim(0, len(y) - 1)
        ax.set_ylabel("D%d" % (i + 1))


plot_signal_decomp(data_constant, 'haar', "DWT: Constant signal")
plot_signal_decomp(data1, 'haar', "DWT: Signal with constant slope")
plot_signal_decomp(data2, 'haar', "DWT: Signal with constant slope except for overlapping values")
plot_signal_decomp(data3, 'haar', "DWT: Signal with constant slope except for sudden jump")
plot_signal_decomp(data4, 'sym5',
                   "DWT: Frequency and phase change - Symmlets5")
plot_signal_decomp(ecg, 'sym5', "DWT: Ecg sample - Symmlets5")


plt.show()

### How to decompose a signal into components containing different frequencies:

#### Create a signal by adding the two above signals: constant low frequency plus a mixed signal (constant in the first part and high frequency in the second part):

In [ ]:
sine_wave_low = signals_lowfreq['200_points']
sine_wave_high = signals_highfreq['200_points']

pylab.subplot(2,2,1)
pylab.plot(sine_wave_low)
pylab.subplot(2,2,2)
pylab.plot(zero_high)
fig.show()

In [ ]:
signal_mixed_half = signals_lowfreq['200_points'] + zero_high

In [ ]:
plt.plot(signal_mixed_half)

#### Task: Decompose the mixed signal (signal_mixed_half) in order to reconstract the original low frequency signal (sine_wave_low):

Visualize frequency composition using discrete wavelet transform: 

In [ ]:
plot_signal_decomp(signal_mixed_half, 'sym14', "Signal decomposition")

#### Decomposing the signal using discrete wavelet transform:
The code below executes the same operation that is visualized above.

In [ ]:
def dwt_steps(a, w, mode, num_steps):
    ca = []
    cd = []
    for i in range(num_steps + 1):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)
    return ca, cd

In [ ]:
w = 'sym14'
mode = pywt.Modes.smooth
ca, cd = dwt_steps(signal_mixed_half, w, mode, 10)

#### Reonstructing the signal from the estimates:

In [ ]:
rec_a = []
rec_d = []

for i, coeff in enumerate(ca):
    coeff_list = [coeff, None] + [None] * i
    rec_a.append(pywt.waverec(coeff_list, w))

for i, coeff in enumerate(cd):
    coeff_list = [None, coeff] + [None] * i
    rec_d.append(pywt.waverec(coeff_list, w))

In [ ]:
low_freq_rec = rec_a[2][:200]

In [ ]:
high_freq_rec = signal_mixed_half - low_freq_rec

#### The two signal components reconstructed by wavelet transform look quite similar to the two original signal components that were used to create the mixed signal: 

In [ ]:
pylab.subplot(2,2,1)
pylab.plot(low_freq_rec)
pylab.subplot(2,2,2)
pylab.plot(high_freq_rec)
fig.show()

In [ ]:
pylab.subplot(2,2,1)
pylab.plot(sine_wave_low)
pylab.subplot(2,2,2)
pylab.plot(zero_high)
fig.show()

### Visualizing different wavelet types: 

In [ ]:
pywt.wavelist()

In [ ]:
w = 'dmey'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'coif5'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'rbio5.5'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'db14'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'rbio3.5'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'sym14'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'bior1.5'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,3,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'coif5'
w = pywt.Wavelet(w)

for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'shan'
w = pywt.ContinuousWavelet(w)
dir(w)
for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'cmor'
w = pywt.ContinuousWavelet(w)
dir(w)
for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'fbsp'
w = pywt.ContinuousWavelet(w)
dir(w)
for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

In [ ]:
w = 'morl'
w = pywt.ContinuousWavelet(w)
dir(w)
for i in range(0, len(w.wavefun())):
    index = i+1
    pylab.subplot(2,2,index)
    pylab.plot(w.wavefun()[i])
fig.show()

## Questions:

Sometimes the function wavefun() shows several functions. In the case of the haar wavelet, the first one is the low-pass filter and the second one is the actual haar wavelet. But there is a third one. What does the third one represent? How do you tell which plot represents what?